In [1]:
import sys
from pathlib import Path

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from src.data_utils import load_raw_data
import pandas as pd

# Cargar los datos
df_raw = load_raw_data(
    fecha_inicio="2023-01-02", 
    fecha_fin="2025-06-30", 
    descargar_bq=False)

df_raw.describe()



Cargando datos desde: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250630.parquet
Total de fechas faltantes: 4
Fechas faltantes: ['2023-12-25T00:00:00.000000000' '2024-01-01T00:00:00.000000000'
 '2024-12-25T00:00:00.000000000' '2025-01-01T00:00:00.000000000']


,fecha,cantidad,base_imponible,tipo_IVA,total,is_summer_peak,is_easter
count,337353,337353.000000,337353.000000,335106.000000,337353.000000,337353.000000,337353.0
mean,2024-03-10 00:54:30.032281344,1.209303,2.010886,9.084558,2.199180,0.208473,0.0
min,2023-01-02 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,2023-07-28 00:00:00,1.000000,1.350000,10.000000,1.450000,0.000000,0.0
50%,2024-03-09 00:00:00,1.000000,1.640000,10.000000,1.800000,0.000000,0.0
75%,2024-09-25 00:00:00,1.000000,2.320000,10.000000,2.550000,0.000000,0.0
max,2025-06-30 00:00:00,93.000000,287.270000,10.000000,316.000000,1.000000,0.0
std,NaN,0.733263,1.692278,2.793367,1.862156,0.406217,0.0


In [4]:
from src.data_utils import transformar_a_series_temporales

ts_data = transformar_a_series_temporales(df_raw)


In [5]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   year            126 non-null    UInt32        
 1   week            126 non-null    UInt32        
 2   familia         126 non-null    object        
 3   base_imponible  126 non-null    float64       
 4   is_summer_peak  126 non-null    int32         
 5   is_easter       126 non-null    int64         
 6   dias_semana     126 non-null    int64         
 7   week_start      126 non-null    datetime64[ns]
dtypes: UInt32(2), datetime64[ns](1), float64(1), int32(1), int64(2), object(1)
memory usage: 6.8+ KB


In [6]:
import os
from dotenv import load_dotenv
from paths import ROOT_DIR
import hopsworks

load_dotenv(os.path.join(ROOT_DIR, '.env'))

HOPSWORKS_PROJECT_NAME = 'fleca_mlops'
HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')  

project = hopsworks.login(
    api_key_value=HOPSWORKS_API_KEY, 
    project=HOPSWORKS_PROJECT_NAME)

feature_store = project.get_feature_store()

2025-08-09 20:35:11,741 INFO: Initializing external client
2025-08-09 20:35:11,741 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-09 20:35:11,741 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-08-09 20:35:12,869 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272


In [7]:
# Creeamos el feature store en Hopsworks
feature_store = project.get_feature_store()

# Definimos el nombre y versión del feature store
FEATURE_GROUP_NAME = "times_series_bolleria_feature_group"
FEATURE_GROUP_VERSION = 1

In [8]:
# Creamos el feature group
# Desactivamos el stream en Kafka
feature_group = feature_store.create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Feature group for time series data of bolleria",
    primary_key=["week_start", "familia"],
    event_time="week_start",
    stream=False # Desactivamos el stream en Kafka
    )


In [9]:
# Comprobar si el feature group existe antes de crearlo
try:
    feature_group = feature_store.get_feature_group(
        name=FEATURE_GROUP_NAME,
        version=FEATURE_GROUP_VERSION
    )
    print(f"Feature group '{FEATURE_GROUP_NAME}' (v{FEATURE_GROUP_VERSION}) ya existe. Usando el existente.")
except Exception as e:
    print(f"No existe el feature group, se creará uno nuevo. Detalle: {e}")
    feature_group = feature_store.create_feature_group(
        name=FEATURE_GROUP_NAME,
        version=FEATURE_GROUP_VERSION,
        description="Feature group for time series data of bolleria",
        primary_key=["week_start", "familia"],
        event_time="week_start",
        stream=False
    )

Feature group 'times_series_bolleria_feature_group' (v1) ya existe. Usando el existente.


In [ ]:
# Forzar year y week a int64 sin nulos antes de la inserción porque me estaba dando problemas
import numpy as np
ts_data['year'] = ts_data['year'].fillna(0).astype(np.int64)
ts_data['week'] = ts_data['week'].fillna(0).astype(np.int64)
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   year            126 non-null    int64         
 1   week            126 non-null    int64         
 2   familia         126 non-null    object        
 3   base_imponible  126 non-null    float64       
 4   is_summer_peak  126 non-null    int32         
 5   is_easter       126 non-null    int64         
 6   dias_semana     126 non-null    int64         
 7   week_start      126 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(4), object(1)
memory usage: 7.5+ KB


In [13]:
# Comprobar el orden temporal antes de la inserción
print(ts_data[['year', 'week', 'week_start']].head(10))
print(ts_data[['year', 'week', 'week_start']].tail(10))
assert ts_data['week_start'].is_monotonic_increasing, 'El DataFrame NO está ordenado por week_start'

   year  week week_start
0  2023     1 2023-01-02
1  2023     2 2023-01-09
2  2023     3 2023-01-16
3  2023     4 2023-01-23
4  2023     5 2023-01-30
5  2023     6 2023-02-06
6  2023     7 2023-02-13
7  2023     8 2023-02-20
8  2023     9 2023-02-27
9  2023    10 2023-03-06
     year  week week_start
116  2025    17 2025-04-21
117  2025    18 2025-04-28
118  2025    19 2025-05-05
119  2025    20 2025-05-12
120  2025    21 2025-05-19
121  2025    22 2025-05-26
122  2025    23 2025-06-02
123  2025    24 2025-06-09
124  2025    25 2025-06-16
125  2025    26 2025-06-23


In [ ]:
feature_group.insert(
    ts_data,
    online=False,
    write_options={'wait_for_job': True}
)

Uploading Dataframe: 100.00% |██████████| Rows 126/126 | Elapsed Time: 00:00 | Remaining Time: 00:00



Launching job: times_series_bolleria_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1242272/jobs/named/times_series_bolleria_feature_group_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1242272/jobs/named/times_series_bolleria_feature_group_1_offline_fg_materialization/executions
2025-08-09 20:36:12,496 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-08-09 20:36:12,496 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-08-09 20:36:15,655 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-08-09 20:36:15,655 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-08-09 20:38:09,464 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Fina

(Job('times_series_bolleria_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)